In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from tensorflow import keras

ImportError: cannot import name 'keras' from 'tensorflow' (unknown location)

In [3]:
train_dir = "../train"
test_dir = "../test"

print(train_dir)
print(test_dir)


../train
../test


In [6]:
# Carregue os datasets
img_size = (64, 64)  # Tamanho desejado (ex: 224x224 para Transfer Learning)
batch_size = 32        # Pode ajustar conforme sua GPU

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,       # Embaralha os dados
    seed=42             # Para reprodutibilidade
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False       # Não precisa embaralhar o teste
)

AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
# Número de classes (letras)
class_names = train_ds.class_names
print("Classes (letras) encontradas:", class_names)

# Formato de um batch de imagens
for images, labels in train_ds.take(1):  # Pega o primeiro batch
    print("\nFormato das imagens:", images.shape)  # (batch_size, 224, 224, 3)
    print("Formato de UMA imagem:", images[0].shape)  # (224, 224, 3)
    print("Rótulos do batch:", labels.numpy())  # Ex: [0, 2, 1, ...] (índices das classes)

In [ ]:
print("GPUs disponíveis:", tf.config.list_physical_devices('GPU'))

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=10)

In [ ]:
# Avaliação no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Acurácia no teste: {test_accuracy:.2%}")
print(f"Perda no teste: {test_loss:.4f}")

In [ ]:
# model.save("modelo_libras.h5")

In [ ]:
#etapa 10 -  TREINAMENTO DE IMAGEM EXTERNA

from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

#caminho das imagens
caminho = './imagens/a.jpg'

#abrir imagem com PIL
img = Image.open(caminho)# L = grayscale

#redimensiona opara 28x28 pixls (formato do dataset Fashion MNIST)
img = img.resize((64,64))

#converter para array numpy e normalizar
img_array = np.array(img) / 255.0  #nomalizar par 0-1(preto ou branco)

#oprcional -  inverte cores se o fundo para escuro se a peça for clara
# img_array = 1 - img_array

#adiciona a dimensao de lote (batch_size=1)
img_array = img_array.reshape(1,64,64) #em 1 dimensao no formato 28x28

import matplotlib.pyplot as plt
plt.imshow(img_array[0], cmap='gray')
plt.title('Imagem tratada')
plt.axis('off')
plt.show()

#se o modelo for treinado com o fundo escuro precisa converter as imagens do mesmo jeito

In [ ]:
#previsao da classe
predicao = model.predict(img_array)

#idetificando o indice da classe cm maior probabilidade
indice_classe = np.argmax(predicao)

print(f'Classe prevista: {classe_nomes[indice_classe]}')
print(f'Confirança: {predicao[0][indice_classe]*100:.2f}%')